In [1]:
#@test {"output": "ignore"}

print 'Installing dependencies...'
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print 'Importing libraries and defining some helper functions...'
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print '🎉 Done!'
print magenta.__version__ 
print tensorflow.__version__

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 130812 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
     |████████████████████████████████| 1.5MB 10.1MB/s 
     |████████████████████████████████| 26.6MB 1.2MB/s 
     |████████████████████████████████| 2.6MB 56.2MB/s 
     |████████████████████████████████| 686kB 53.5MB/s 
     |████████████████████████████████| 204kB 55.9MB/s 
     |████████████████████████████████| 2.3MB 41.1MB/s 
     |████████████████████████████

In [0]:
# import ctypes.util
# orig_ctypes_util_find_library = ctypes.util.find_library
# def proxy_find_library(lib):
#   if lib == 'fluidsynth':
#     return 'libfluidsynth.so.1'
#   else:
#     return orig_ctypes_util_find_library(lib)
# ctypes.util.find_library = proxy_find_library

# from google.colab import files

# import magenta.music as mm
# import magenta
# import tensorflow

# print '🎉 Done!'
# print magenta.__version__ 
# print tensorflow.__version__

In [0]:
import numpy as np

In [0]:
new_songs_raw = np.load('new_songs.npy')

In [0]:
from magenta.protobuf import music_pb2

magenta_songs = []

for k in range(new_songs_raw.shape[0]):
  song = music_pb2.NoteSequence()

  st_time = 0
  end_time = 0
  
  for m in range(new_songs_raw.shape[1]):
    # Add the notes to the sequence.
    
    pitch = int(new_songs_raw[k,m,0])
    st_time = end_time
    end_time = st_time + new_songs_raw[k,m,2]
    veloc = int(new_songs_raw[k,m,1])
    
    song.notes.add(pitch=pitch, 
                   start_time=st_time, 
                   end_time=end_time, 
                   velocity=veloc)
  
  magenta_songs.append(song)


In [8]:
for k in range(len(magenta_songs)):
  magenta_songs[k].total_time = 120
  magenta_songs[k].tempos.add(qpm=100);
  # This is a colab utility method that visualizes a NoteSequence.
  mm.plot_sequence(magenta_songs[k])

  # This is a colab utility method that plays a NoteSequence.
  # mm.play_sequence(magenta_songs[k],synth=mm.fluidsynth)
  mm.play_sequence(magenta_songs[k],synth=mm.synthesize)


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

In [0]:
# """ This module prepares midi file data and feeds it to the neural
#     network for training """
# import glob
# import pickle
# import numpy
# from music21 import converter, instrument, note, chord
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.layers import LSTM
# from keras.layers import Activation
# from keras.utils import np_utils
# from keras.callbacks import ModelCheckpoint

In [0]:
# from music21 import converter, instrument, note, chord
# notes = []
# for file in glob.glob("*.mid"):
#     midi = converter.parse(file)
#     notes_to_parse = None
#     parts = instrument.partitionByInstrument(midi)
#     if parts: # file has instrument parts
#         notes_to_parse = parts.parts[0].recurse()
#     else: # file has notes in a flat structure
#         notes_to_parse = midi.flat.notes
#     for element in notes_to_parse:
#         if isinstance(element, note.Note):
#             notes.append(str(element.pitch))
#         elif isinstance(element, chord.Chord):
#             notes.append('.'.join(str(n) for n in element.normalOrder))

In [0]:
# notes